<a href="https://www.kaggle.com/code/avikumart/q-a-modeling-using-nlp-models?scriptVersionId=236629475" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split, cross_val_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Load the dataset

In [5]:
# load the dataset
df1 = pd.read_csv("/kaggle/input/questionanswer-dataset/S08_question_answer_pairs.txt", sep='\t')
df1.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [9]:
df2 = pd.read_csv("/kaggle/input/questionanswer-dataset/S09_question_answer_pairs.txt", sep="\t")
df2.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,S09_set4_a10
1,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,S09_set4_a10
2,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,S09_set4_a10
3,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,S09_set4_a10
4,Alessandro_Volta,Did Volta have a passion for the study of elec...,yes,easy,medium,S09_set4_a10


In [12]:
df3 = pd.read_csv("/kaggle/input/questionanswer-dataset/S10_question_answer_pairs.txt", sep="\t", encoding = 'ISO-8859-1')
df3.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,Alessandro Volta was not a professor of chemis...,easy,easy,S10_set4_a10
1,Alessandro_Volta,Was Alessandro Volta a professor of chemistry?,No,easy,hard,S10_set4_a10
2,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Alessandro Volta did invent the remotely opera...,easy,easy,S10_set4_a10
3,Alessandro_Volta,Did Alessandro Volta invent the remotely opera...,Yes,easy,easy,S10_set4_a10
4,Alessandro_Volta,Was Alessandro Volta taught in public schools?,Volta was taught in public schools.,easy,easy,S10_set4_a10


## Analyze and prepare the dataset

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1715 entries, 0 to 1714
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              1715 non-null   object
 1   Question                  1696 non-null   object
 2   Answer                    1473 non-null   object
 3   DifficultyFromQuestioner  1224 non-null   object
 4   DifficultyFromAnswerer    1473 non-null   object
 5   ArticleFile               1713 non-null   object
dtypes: object(6)
memory usage: 80.5+ KB


In [14]:
df1["ArticleTitle"].value_counts()

ArticleTitle
Calvin_Coolidge       77
Finland               76
Singapore             71
James_Monroe          66
Uruguay               65
kangaroo              63
turtle                62
otter                 62
Amedeo_Avogadro       61
Canada                60
leopard               59
Ghana                 58
Millard_Fillmore      55
Abraham_Lincoln       54
elephant              53
Liechtenstein         52
duck                  52
penguin               50
Romania               48
polar_bear            48
beetle                48
Gerald_Ford           47
John_Adams            46
Woodrow_Wilson        46
Grover_Cleveland      41
Theodore_Roosevelt    36
Gray_Wolf             36
Egypt                 35
Qatar                 34
Indonesia             33
Ulysses_S._Grant      32
Alessandro_Volta      24
Isaac_Newton          24
Anders_Celsius        17
Henri_Becquerel       13
James_Watt             7
Nikola_Tesla           3
\n                     1
Name: count, dtype: int64

In [15]:
df2["ArticleTitle"].value_counts()

ArticleTitle
Michael_Faraday                49
Charles-Augustin_de_Coulomb    36
Swan                           36
Henri_Becquerel                36
Alessandro_Volta               36
Cymbal                         35
Amedeo_Avogadro                35
Cello                          32
Turtle                         32
Ottawa                         28
Anders_Celsius                 26
Chinese_language               26
Santiago                       20
Spanish_language               20
Italian_language               19
Otter                          18
Swahili_language               18
Isaac_Newton                   18
Violin                         18
Nikola_Tesla                   17
Japanese_language              17
Flute                          17
Tiger                          17
Fox                            17
Giraffe                        17
French_language                16
Lima                           15
Blaise_Pascal                  15
Arabic_language                14
L

In [16]:
data = pd.concat([df1,df2,df3], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3998 entries, 0 to 3997
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              3998 non-null   object
 1   Question                  3961 non-null   object
 2   Answer                    3420 non-null   object
 3   DifficultyFromQuestioner  3043 non-null   object
 4   DifficultyFromAnswerer    3418 non-null   object
 5   ArticleFile               3996 non-null   object
dtypes: object(6)
memory usage: 187.5+ KB


In [17]:
data.isnull().sum()

ArticleTitle                  0
Question                     37
Answer                      578
DifficultyFromQuestioner    955
DifficultyFromAnswerer      580
ArticleFile                   2
dtype: int64

In [129]:
qad = data[["ArticleTitle","Question","Answer"]]
qad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3998 entries, 0 to 3997
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ArticleTitle  3998 non-null   object
 1   Question      3961 non-null   object
 2   Answer        3420 non-null   object
dtypes: object(3)
memory usage: 93.8+ KB


In [130]:
qad = qad.dropna()

In [131]:
qad = qad.drop_duplicates()

In [132]:
qad.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3112 entries, 0 to 3997
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ArticleTitle  3112 non-null   object
 1   Question      3112 non-null   object
 2   Answer        3112 non-null   object
dtypes: object(3)
memory usage: 97.2+ KB


In [133]:
qad.head(15)

,ArticleTitle,Question,Answer
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.
4,Abraham_Lincoln,Did his mother die of pneumonia?,no
5,Abraham_Lincoln,Did his mother die of pneumonia?,No.
6,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months
7,Abraham_Lincoln,How many long was Lincoln's formal education?,18 months.
8,Abraham_Lincoln,When did Lincoln begin his political career?,1832
9,Abraham_Lincoln,When did Lincoln begin his political career?,1832.


In [134]:
qad["ArticleTitle"].value_counts()

ArticleTitle
Amedeo_Avogadro                78
Alessandro_Volta               71
Michael_Faraday                64
Charles-Augustin_de_Coulomb    59
Uruguay                        59
                               ..
Michelangelo                    7
Vincent_van_Gogh                7
Japanese_language               3
Dragonfly                       3
\n                              1
Name: count, Length: 107, dtype: int64

## Feature encoding and Model architecture design

In [135]:
# unique article categories
qad["ArticleTitle"].nunique()

107

In [136]:
X = qad[["ArticleTitle","Question"]]
y = qad["Answer"]

In [137]:
# tokenizer for the question and answer
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [179]:
# reference col tokenization
reftoken = Tokenizer()
reftoken.fit_on_texts(X_train["ArticleTitle"])
train_ref = reftoken.texts_to_sequences(X_train["ArticleTitle"])
train_padref = pad_sequences(train_ref)

test_ref = reftoken.texts_to_sequences(X_test["ArticleTitle"])
test_padref = pad_sequences(test_ref)

In [180]:
# question tokenization
qtoken = Tokenizer(num_words=100000,oov_token='<00V>')
qtoken.fit_on_texts(X_train["Question"])
train_seq = qtoken.texts_to_sequences(X_train["Question"])
train_padseq  = pad_sequences(train_seq)

# encode the test data
test_seq = qtoken.texts_to_sequences(X_test["Question"])
test_padseq = pad_sequences(test_seq)

In [172]:
# tokenize the answer
atoken = Tokenizer()
atoken.fit_on_texts(y_train)
ytrain_seq = atoken.texts_to_sequences(y_train)
y_train_pad = pad_sequences(ytrain_seq, maxlen=3132)

ytest_seq = atoken.texts_to_sequences(y_test)
y_test_pad = pad_sequences(ytest_seq,maxlen=3132)

In [181]:
# input lengths 
ref_vocab = len(reftoken.word_index)
question_vocab = len(qtoken.word_index)
answer_vocab = len(atoken.word_index)

In [164]:
# keras layers for the model design
from keras.layers import Embedding, Flatten, Dense, Dropout, Input, concatenate
from keras.layers import Conv1D, SimpleRNN, Bidirectional, MaxPooling1D, GlobalMaxPool1D, LSTM, GRU
from keras.models import Sequential
from keras.regularizers import L1L2
from keras import Model

In [182]:
# take the inputs from article title and and question
que_ref = Input(shape=(None,))
ref_embed = Embedding(input_dim=ref_vocab, output_dim=70)(que_ref)
ref_output = LSTM(10, activation="relu")(ref_embed)

# encode the question input
que_input = Input(shape=(None,))
que_embed = Embedding(input_dim=question_vocab, output_dim=70)(que_input)
que_encode = LSTM(16, activation="tanh", return_sequences=True)(que_embed)
que_encode1 = LSTM(16, activation="tanh", dropout=0.20)(que_encode)

concate_ques = concatenate([ref_output,que_encode1])

# decode using the answer output
answer = Dense(answer_vocab, activation="softmax")(concate_ques)

model = Model([que_ref,que_input], answer)

In [183]:
model.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_24            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_23            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_19 (Embedding)  │ (None, None, 70)       │        268,240 │ input_layer_24[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_18 (Embedding)  │ (None, None, 70)       │          9,940 │ input_layer_23[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_31 (LSTM)            │ (None, None, 16)       │          5,568 │ embedding_19[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_30 (LSTM)            │ (None, 10)             │          3,240 │ embedding_18[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_32 (LSTM)            │ (None, 16)             │          2,112 │ lstm_31[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_9             │ (None, 26)             │              0 │ lstm_30[0][0],         │
│ (Concatenate)             │                        │                │ lstm_32[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 3132)           │         84,564 │ concatenate_9[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 373,664 (1.43 MB)

 Trainable params: 373,664 (1.43 MB)

 Non-trainable params: 0 (0.00 B)

In [184]:
# compile the model and fit to the training data
model.compile(optimizer="adam",
             loss="categorical_crossentropy")

In [186]:
model.fit([train_padref,train_padseq], y_train_pad, epochs=20, batch_size=64)

Epoch 1/20


I0000 00:00:1745829068.670305      97 cuda_dnn.cc:529] Loaded cuDNN version 90300


39/39 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 20215.6699
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 14485.9854
Epoch 3/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 28840.9922
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 212279.4531
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1292622.1250
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 4926118.5000
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 20226824.0000
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 56075552.0000
Epoch 9/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 116567656.0000
Epoch 10/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 256050816.0000
Epoch 11/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 603523712.0000
Epoch 12/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1136039936.0000
Epoch 13/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1757975680.0000
Epoch 14/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3618

In [187]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, concatenate

# Example Data
X_train = pd.DataFrame({
    "ArticleTitle": ["Deep Learning Intro", "Neural Networks Basics", "Advanced LSTM Models"],
    "Question": ["What is deep learning?", "Explain neural networks", "How to use LSTM?"]
})

y_train = np.array([0, 1, 0])  # Example labels (binary classification)

# Step 1: Tokenize and pad both inputs
# Tokenizer for ArticleTitle
title_tokenizer = Tokenizer()
title_tokenizer.fit_on_texts(X_train["ArticleTitle"])
title_seq = title_tokenizer.texts_to_sequences(X_train["ArticleTitle"])
title_padseq = pad_sequences(title_seq, maxlen=10)

# Tokenizer for Question
question_tokenizer = Tokenizer()
question_tokenizer.fit_on_texts(X_train["Question"])
question_seq = question_tokenizer.texts_to_sequences(X_train["Question"])
question_padseq = pad_sequences(question_seq, maxlen=10)

# Step 2: Define Inputs
input_title = Input(shape=(10,), name="Title_Input")
input_question = Input(shape=(10,), name="Question_Input")

# Step 3: Embedding Layers
embed_title = Embedding(input_dim=len(title_tokenizer.word_index)+1, output_dim=8)(input_title)
embed_question = Embedding(input_dim=len(question_tokenizer.word_index)+1, output_dim=8)(input_question)

# Step 4: LSTM layers
lstm_title = LSTM(16)(embed_title)
lstm_question = LSTM(16)(embed_question)

# Step 5: Merge
merged = concatenate([lstm_title, lstm_question])

# Step 6: Dense layers
dense1 = Dense(32, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense1)  # Binary classification

# Step 7: Model
model = Model(inputs=[input_title, input_question], outputs=output)

# Step 8: Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 9: Fit
model.fit([title_padseq, question_padseq], y_train, epochs=10, batch_size=2)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7778 - loss: 0.6939
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7778 - loss: 0.6897
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7778 - loss: 0.6886 
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6111 - loss: 0.6896
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.6111 - loss: 0.6889
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7778 - loss: 0.6861
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6111 - loss: 0.6861
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6111 - loss: 0.6845 
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6111 - loss: 0.6825
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6111 - loss: 0.6815
